#Twinder: Sistema de clasificación

Hemos entrenado 2 SVMs a partir de 2 datasets distintos. El primero de ellos se trata de reviews de usuarios en Amazon.com, y están etiquetados en 2 posibles valores (positivo y negativo).


El segundo de los datasets, conocido como sentiment140, consiste en publicaciones de twitter con 2 etiquetas.

# Primer DS


In [ ]:
import pandas as pd
import os

from nltk.tokenize import word_tokenize 
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

import nltk

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import svm
from sklearn.metrics import classification_report


#Max 3600000
trainingValues=10000

#Max 400000
testingValues=100

In [ ]:
f = open("/content/drive/MyDrive/train.ft.txt", "r")
lines = f.readlines()
trainingData_aws = pd.DataFrame(columns=['label','text'])
print(len(lines))
i=0
for line in lines:
  i+=1
  #print("\t",i)
  label = line.split(' ',1)[0]
  if label == "__label__1":
    label="0"
  elif label == "__label__2":
    label="4"
  #print(label)
  text = line.split(' ',1)[1]
  #print(text)
  trainingData_aws = trainingData_aws.append({'label':label,'text':text},ignore_index=True)
  if i==trainingValues:
    break;

len(trainingData_aws.index)
print(trainingData_aws)

3600000
     label                                               text
0        4  Stuning even for the non-gamer: This sound tra...
1        4  The best soundtrack ever to anything.: I'm rea...
2        4  Amazing!: This soundtrack is my favorite music...
3        4  Excellent Soundtrack: I truly like this soundt...
4        4  Remember, Pull Your Jaw Off The Floor After He...
...    ...                                                ...
9995     4  A revelation of life in small town America in ...
9996     4  Great biography of a very interesting journali...
9997     0  Interesting Subject; Poor Presentation: You'd ...
9998     0  Don't buy: The box looked used and it is obvio...
9999     4  Beautiful Pen and Fast Delivery.: The pen was ...

[10000 rows x 2 columns]


Una vez creado el Panda.DataFrame introduciendo los datos del dataset en el formato desado, debemos eliminar stopwords y pasar a forma raiz los verbos conjugados y palabras en plural...

In [ ]:
nltk.download('punkt')
nltk.download('stopwords')

ps = PorterStemmer()

preprocessedText = []

for row in trainingData_aws.itertuples():
    
    
    text = word_tokenize(row[2]) ## indice de la columna que contiene el texto
    ## Remove stop words
    stops = set(stopwords.words("english"))
    text = [ps.stem(w) for w in text if not w in stops and w.isalnum()]
    text = " ".join(text)
    
    preprocessedText.append(text)

preprocessedData_aws = trainingData_aws
preprocessedData_aws['processed_text'] = preprocessedText

len(preprocessedData_aws.index)
print(preprocessedData_aws)
preprocessedData_aws['label'].value_counts()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
     label  ...                                     processed_text
0        4  ...  stune even thi sound track beauti It paint sen...
1        4  ...  the best soundtrack ever anyth I read lot revi...
2        4  ...  amaz thi soundtrack favorit music time hand th...
3        4  ...  excel soundtrack I truli like soundtrack I enj...
4        4  ...  rememb pull your jaw off the floor after hear ...
...    ...  ...                                                ...
9995     4  ...  A revel life small town america earli 1900 tho...
9996     4  ...  great biographi interest journalist thi biogra...
9997     0  ...  interest subject poor present you tell bore st...
9998     0  ...  Do buy the box look use obvious new I tri cont...
9999     4  ...  beauti pen fast deliveri the 

0    5097
4    4903
Name: label, dtype: int64

A continuación creamos el Bag of Words a partir del conjunto de datos ya procesado.

In [ ]:

bagOfWordsModel_aws = TfidfVectorizer()
bagOfWordsModel_aws.fit(preprocessedData_aws['processed_text'])
textsBoW_aws= bagOfWordsModel_aws.transform(preprocessedData_aws['processed_text'])
print("Finished")

print(textsBoW_aws.shape)

Finished
(10000, 21397)


Ahora entrenamos con Support Vector Machine.

In [ ]:
svc_aws = svm.SVC(kernel='linear') #Modelo de clasificación

X_train_aws = textsBoW_aws #Documentos
Y_train_aws = trainingData_aws['label'] #Etiquetas de los documentos 
svc_aws.fit(X_train_aws, Y_train_aws) #Entrenamiento

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

Ahora importamos el dataset de test (prueba) como hicimos con el anterior.

In [ ]:

f = open("/content/drive/MyDrive/test.ft.txt", "r")
lines = f.readlines()
testingData_aws = pd.DataFrame(columns=['label','text'])
print(len(lines))
i=0
for line in lines:
  i+=1
  #print("\t",i)
  label = line.split(' ',1)[0]
  if label == "__label__1":
    label="0"
  elif label == "__label__2":
    label="4"
  #print(label)
  text = line.split(' ',1)[1]
  #print(text)
  testingData_aws = testingData_aws.append({'label':label,'text':text},ignore_index=True)
  if i==testingValues:
    break;

len(testingData_aws.index)
print(testingData_aws)

400000
   label                                               text
0      4  Great CD: My lovely Pat has one of the GREAT v...
1      4  One of the best game music soundtracks - for a...
2      0  Batteries died within a year ...: I bought thi...
3      4  works fine, but Maha Energy is better: Check o...
4      4  Great for the non-audiophile: Reviewed quite a...
..   ...                                                ...
95     4  The Scarlet Letter a must read for any America...
96     0  The Scarlet Letter: A soap opera: Although thi...
97     4  This book is Great!: One might notice that man...
98     0  I had to read it...: I had to read this book f...
99     4  Possibly the funniest movie ever made.: It sta...

[100 rows x 2 columns]


Seguidamente eliminamos stopwords y pasamos los datos por el stem para convertir las palabras a su raiz.

In [ ]:
ps = PorterStemmer()

preprocessedText = []

for row in testingData_aws.itertuples():
    
    
    text = word_tokenize(row[2]) ## indice de la columna que contiene el texto
    ## Remove stop words
    stops = set(stopwords.words("english"))
    text = [ps.stem(w) for w in text if not w in stops and w.isalnum()]
    text = " ".join(text)
    
    preprocessedText.append(text)

preprocessedDataTest_aws = testingData_aws
preprocessedDataTest_aws['processed_text'] = preprocessedText

print(preprocessedDataTest_aws)
print(testingData_aws['label'].value_counts())

   label  ...                                     processed_text
0      4  ...  great CD My love pat one great voic gener I li...
1      4  ...  one best game music soundtrack game I realli p...
2      0  ...  batteri die within year I bought charger jul 2...
3      4  ...  work fine maha energi better check maha energi...
4      4  ...  great review quit bit combo player hesit due u...
..   ...  ...                                                ...
95     4  ...  the scarlet letter must read american the best...
96     0  ...  the scarlet letter A soap opera although book ...
97     4  ...  thi book great one might notic mani neg review...
98     0  ...  I read I read book school I like I sure point ...
99     4  ...  possibl funniest movi ever made It start bit s...

[100 rows x 3 columns]
4    53
0    47
Name: label, dtype: int64


Creamos el Bag of Words con los textos de tests

In [ ]:
textsBoWTest_aws= bagOfWordsModel_aws.transform(preprocessedDataTest_aws['processed_text'])
print("Finished")
print(textsBoWTest_aws.shape)

Finished
(100, 21397)


Generamos el array (ndarray, array de numPy) con las predicciones del svc entrenado anteriormente. Seguidamente empleamos el método classification_report para comparar las predicciones con las etiquetas reales.

In [ ]:

X_test_aws = textsBoWTest_aws #Documentos
predictions_aws = svc_aws.predict(X_test_aws) #Se almacena en el array predictions las predicciones del clasificador
Y_test_aws = testingData_aws['label'] #Etiquetas reales de los documentos

print (classification_report(Y_test_aws, predictions_aws))


              precision    recall  f1-score   support

           0       0.90      0.91      0.91        47
           4       0.92      0.91      0.91        53

    accuracy                           0.91       100
   macro avg       0.91      0.91      0.91       100
weighted avg       0.91      0.91      0.91       100



# Segundo DS


Para el segundo dataset repetiremos los pasos de la sección anterior con pequeñas diferencias:

*   Importamos el .csv de la carpeta de drive
*   Restringimos el tamaño del dataset para realizar pruebas.
*   Agregamos el nombre de las columnas, y eliminamos las columnas innecesarias para quedarnos unicamente con el texto y la etiqueta.
*   Procesamos los textos (tweets) eliminando stopwords y transformado las palabras a su forma raiz.
*   Generamos el BagOfWords a partir del texto procesado.
*   Entrenamos la svm con el nombre **svc_s140**.
*   Repetimos los primeros 6 pasos para el dataset de prueba.
*   Hacemos las predicciones con el dataset de prueba y la svc_s140. Seguidamente mostramos las predicciones para comprobar, entre otros, el accuracy obtenido.







In [ ]:
trainingData_s140 = pd.read_csv('/content/drive/MyDrive/sentiment140.csv', encoding='ISO-8859-1')
trainingData_s140 = trainingData_s140.head(trainingValues//2).append(trainingData_s140.tail(trainingValues//2))
trainingData_s140 = trainingData_s140.set_axis(['label','id','date','flag','user','text'], axis=1, inplace=False)
trainingData_s140 = trainingData_s140.drop(['id','date','flag','user'],axis=1)
print(trainingData_s140)

         label                                               text
0            0  is upset that he can't update his Facebook by ...
1            0  @Kenichan I dived many times for the ball. Man...
2            0    my whole body feels itchy and like its on fire 
3            0  @nationwideclass no, it's not behaving at all....
4            0                      @Kwesidei not the whole crew 
...        ...                                                ...
1599994      4  Just woke up. Having no school is the best fee...
1599995      4  TheWDB.com - Very cool to hear old Walt interv...
1599996      4  Are you ready for your MoJo Makeover? Ask me f...
1599997      4  Happy 38th Birthday to my boo of alll time!!! ...
1599998      4  happy #charitytuesday @theNSPCC @SparksCharity...

[10000 rows x 2 columns]


In [ ]:

index_label = trainingData_s140[ trainingData_s140['label'] == "2" ].index

trainingData_s140.drop(index_label, inplace = True)

print(trainingData_s140)

         label                                               text
0            0  is upset that he can't update his Facebook by ...
1            0  @Kenichan I dived many times for the ball. Man...
2            0    my whole body feels itchy and like its on fire 
3            0  @nationwideclass no, it's not behaving at all....
4            0                      @Kwesidei not the whole crew 
...        ...                                                ...
1599994      4  Just woke up. Having no school is the best fee...
1599995      4  TheWDB.com - Very cool to hear old Walt interv...
1599996      4  Are you ready for your MoJo Makeover? Ask me f...
1599997      4  Happy 38th Birthday to my boo of alll time!!! ...
1599998      4  happy #charitytuesday @theNSPCC @SparksCharity...

[10000 rows x 2 columns]


In [ ]:
preprocessedText = []

for row in trainingData_s140.itertuples():
    
    
    text = word_tokenize(row[2]) ## indice de la columna que contiene el texto
    ## Remove stop words
    #stops = set(stopwords.words("english"))
    text = [ps.stem(w) for w in text if not w in stops and w.isalnum()]
    text = " ".join(text)
    
    preprocessedText.append(text)

preprocessedData_s140 = trainingData_s140
preprocessedData_s140['processed_text'] = preprocessedText

len(preprocessedData_s140.index)
print(preprocessedData_s140)
preprocessedData_s140['label'].value_counts()

         label  ...                                     processed_text
0            0  ...  upset ca updat facebook text might cri result ...
1            0  ...  kenichan I dive mani time ball manag save 50 t...
2            0  ...                    whole bodi feel itchi like fire
3            0  ...                 nationwideclass behav mad I ca see
4            0  ...                                kwesidei whole crew
...        ...  ...                                                ...
1599994      4  ...               just woke have school best feel ever
1599995      4  ...             veri cool hear old walt interview http
1599996      4  ...                   are readi mojo makeov ask detail
1599997      4  ...  happi 38th birthday boo alll time tupac amaru ...
1599998      4  ...  happi charitytuesday thenspcc sparkschar speak...

[10000 rows x 3 columns]


4    5000
0    5000
Name: label, dtype: int64

In [ ]:
bagOfWordsModel_s140 = TfidfVectorizer()
bagOfWordsModel_s140.fit(preprocessedData_s140['processed_text'])
textsBoW_s140= bagOfWordsModel_s140.transform(preprocessedData_s140['processed_text'])
print("Finished")

print(textsBoW_s140.shape)

Finished
(10000, 14369)


Ahora entrenamos la svm (Support Vector Machine)

In [ ]:
svc_s140 = svm.SVC(kernel='linear') #Modelo de clasificación

X_train_s140 = textsBoW_s140 #Documentos
Y_train_s140 = trainingData_s140['label'] #Etiquetas de los documentos 
svc_s140.fit(X_train_s140, Y_train_s140) #Entrenamiento

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

Ahora creamos el dataframe con los datos de test

In [ ]:
testingData_s140 = pd.read_csv('/content/drive/MyDrive/sentiment140.csv', encoding='ISO-8859-1')
#testingData_s140 = testingData_s140.head(testingValues//2).append(testingData_s140.tail(testingValues//2))
#testingData_s140 = testingData_s140.iloc[trainingValues//2:trainingValues//2+testingValues//2].append(testingData_s140.iloc[trainingValues//2+len(testingData_s140):trainingValues//2+testingValues//2]+len(testingData_s140))
testingData_s140 = testingData_s140.iloc[trainingValues//2:trainingValues//2+testingValues//2].append(testingData_s140.iloc[1599449:1599499])

testingData_s140 = testingData_s140.set_axis(['label','id','date','flag','user','text'], axis=1, inplace=False)
testingData_s140 = testingData_s140.drop(['id','date','flag','user'],axis=1)

print(testingData_s140)

         label                                               text
5000         0         Orange juice and toothpaste mouth combo = 
5001         0  And hating that watching House will kill me! I...
5002         0  Sitting in my car, waiting for my mum to come ...
5003         0  @zampeachie @Farscale To answer both of you: I...
5004         0  Is home! Been @ WCH since 8:30AM this morning....
...        ...                                                ...
1599494      4                    @Disney_Dreaming Mitchel it is 
1599495      4  just watched twilight...it was horrible...now ...
1599496      4  @nayes1982 The changing them green in solidari...
1599497      4  SUMMEEEEEEER!    Lines, Vines &amp; Trying tim...
1599498      4  On the way to Walsrode! Tomorrow will be one o...

[100 rows x 2 columns]


In [ ]:
ps = PorterStemmer()

preprocessedText = []

for row in testingData_s140.itertuples():
    
    
    text = word_tokenize(row[2]) ## indice de la columna que contiene el texto
    ## Remove stop words
    stops = set(stopwords.words("english"))
    text = [ps.stem(w) for w in text if not w in stops and w.isalnum()]
    text = " ".join(text)
    
    preprocessedText.append(text)

preprocessedDataTest_s140 = testingData_s140
preprocessedDataTest_s140['processed_text'] = preprocessedText

print(preprocessedDataTest_s140)
print(preprocessedDataTest_s140['label'].value_counts())

         label  ...                                     processed_text
5000         0  ...                   orang juic toothpast mouth combo
5001         0  ...  and hate watch hous kill I ca wait next episod...
5002         0  ...                  sit car wait mum come home I lock
5003         0  ...  zampeachi farscal To answer I studi four hour ...
5004         0  ...  Is home been wch sinc morn been awak sinc 4 2n...
...        ...  ...                                                ...
1599494      4  ...                                            mitchel
1599495      4  ...  watch twilight horribl I watch princess diari ...
1599496      4  ...  nayes1982 the chang green solidar iranian iran...
1599497      4  ...  summeeeeeeer line vine amp tri time mine sinc ...
1599498      4  ...           On way walsrod tomorrow one greatest day

[100 rows x 3 columns]
4    50
0    50
Name: label, dtype: int64


In [ ]:
textsBoWTest_s140 = bagOfWordsModel_s140.transform(preprocessedDataTest_s140['processed_text'])
print("Finished")
print(textsBoWTest_s140.shape)



Finished
(100, 14369)


Generamos las predicciones con la svc entrenada anteriormente.

In [ ]:
X_test_s140 = textsBoWTest_s140 #Documentos

predictions_s140 = svc_s140.predict(X_test_s140) #Se almacena en el array predictions las predicciones del clasificador

Y_test_s140 = testingData_s140['label'] #Etiquetas reales de los documentos

print(Y_test_s140)
print(40*'*')
print(predictions_s140)

print(classification_report(Y_test_s140, predictions_s140))

5000       0
5001       0
5002       0
5003       0
5004       0
          ..
1599494    4
1599495    4
1599496    4
1599497    4
1599498    4
Name: label, Length: 100, dtype: int64
****************************************
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 4 0 0 0 4 0 0 0 0 4 4 4 0 4 0 0 0 0
 0 0 0 0 0 0 0 4 0 0 0 0 0 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
 4 0 4 4 4 4 4 4 4 4 0 4 4 0 4 4 4 4 4 0 4 4 0 4 4 4]
              precision    recall  f1-score   support

           0       0.90      0.86      0.88        50
           4       0.87      0.90      0.88        50

    accuracy                           0.88       100
   macro avg       0.88      0.88      0.88       100
weighted avg       0.88      0.88      0.88       100



#Pruebas cruzadas

A continuación pasaremos a testear las dos SVMs con los datasets que NO fueron entrenados.
Para el primer ejemplo hemos empleado el dataset de amazon reviews, que contiene las reviews de productos etiquetadas con positivo o negativo en función de como haya sido su review.

Para el segundo dataset hemos empleado el dataset de sentiment140, empleado en las clases de teoría, que contiene millones de tweets etiquetados en función de su connotación (negativa, neutra o positiva). Nosotros hemos eliminado las publicaciones neutras para que se adecuara al entrenamiento del primer caso y así poder despues compararlos.

Nombres del dataset de Amazon Reviews:


*   trainingData_aws : DataFrame del dataset de entrenamiento de amazon reviews
*   testingData_aws : DataFrame del dataset de pruebas de amazon reviews
*   TextsBoW_aws : Bag Of Words del dataset de entrenamiento de amazon reviews
*   TextsBoWTest_aws : Bag Of Words del dataset de pruebas de amazon reviews
*   svc_aws : SVM entrenada con el dataset de amazon reviews
*   X_train_aws : Textos del dataset de entrenamiento de amazon reviews
*   Y_train_aws : Etiquetas dataset de entrenamiento de amazon reviews
*   X_test_aws : Textos del dataset de pruebas de amazon reviews
*   Y_test_aws : Etiquetas del dataset de pruebas de amazon reviews

Nombres del dataset de Sentiment 140:
*   trainingData_s140 : DataFrame del dataset de entrenamiento de sentiment 140
*   testingData_s140 : DataFrame del dataset de pruebas de sentiment 140
*   TextsBoW_s140 : Bag Of Words del dataset de entrenamiento de sentiment 140
*   TextsBoWTest_s140 : Bag Of Words del dataset de pruebas de sentiment 140
*   svc_s140 : SVM entrenada con el dataset de sentiment 140
*   X_train_s140 : Textos del dataset de entrenamiento de sentiment 140
*   Y_train_s140 : Etiquetas dataset de entrenamiento de sentiment 140
*   X_test_s140 : Textos del dataset de pruebas de sentiment 140
*   Y_test_s140 : Etiquetas del dataset de pruebas de sentiment 140





Ahora probamos el dataset s140 con el algoritmo entrenado con aws

In [ ]:
X_test_aws_s140 = bagOfWordsModel_s140.transform(preprocessedDataTest_aws['processed_text'])

predictions_aws_s140 = svc_s140.predict(X_test_aws_s140) #Se almacena en el array predictions las predicciones del clasificador

Y_test_aws_s140 = testingData_aws['label'].astype(int)

print(X_test_aws_s140)
print(40*'*')
print(predictions_aws_s140)

print(classification_report(Y_test_aws_s140, predictions_aws_s140))

  (0, 1249)	0.08412300838529507
  (0, 1512)	0.08596312104045367
  (0, 1520)	0.15590686996657177
  (0, 1538)	0.10132059579237923
  (0, 1604)	0.11818723396830796
  (0, 1747)	0.10046571854164117
  (0, 2308)	0.44618054309700683
  (0, 3432)	0.15590686996657177
  (0, 4245)	0.15590686996657177
  (0, 4259)	0.11991318985645062
  (0, 4270)	0.1492647717725155
  (0, 4485)	0.0732355973225933
  (0, 4496)	0.13319738624456315
  (0, 4993)	0.1492647717725155
  (0, 5002)	0.12403007951438783
  (0, 5167)	0.06299789427872415
  (0, 5256)	0.159493076905487
  (0, 6299)	0.15590686996657177
  (0, 6317)	0.07421226313191584
  (0, 6988)	0.10724723400081589
  (0, 7389)	0.18637649960456082
  (0, 7400)	0.06718340158193163
  (0, 7477)	0.09610386835996934
  (0, 7611)	0.13315190145720263
  (0, 7702)	0.1379100299115112
  :	:
  (98, 1747)	0.21565539421544275
  (98, 3691)	0.39707468034611687
  (98, 7400)	0.14421300283519764
  (98, 9599)	0.18760674288717957
  (98, 9871)	0.23401749101245528
  (98, 10359)	0.3993074820508697
  

Ahora probamos el dataset aws con el algotimo que hemos entrenado con s140

In [ ]:
X_test_s140_aws = bagOfWordsModel_aws.transform(preprocessedDataTest_s140['processed_text'])

predictions_s140_aws = svc_aws.predict(X_test_s140_aws) #Se almacena en el array predictions las predicciones del clasificador

Y_test_s140_aws = testingData_s140['label'].astype(str)

print(X_test_aws_s140)
print(40*'*')
print(predictions_aws_s140)

print(classification_report(Y_test_s140_aws, predictions_s140_aws))

  (0, 1249)	0.08412300838529507
  (0, 1512)	0.08596312104045367
  (0, 1520)	0.15590686996657177
  (0, 1538)	0.10132059579237923
  (0, 1604)	0.11818723396830796
  (0, 1747)	0.10046571854164117
  (0, 2308)	0.44618054309700683
  (0, 3432)	0.15590686996657177
  (0, 4245)	0.15590686996657177
  (0, 4259)	0.11991318985645062
  (0, 4270)	0.1492647717725155
  (0, 4485)	0.0732355973225933
  (0, 4496)	0.13319738624456315
  (0, 4993)	0.1492647717725155
  (0, 5002)	0.12403007951438783
  (0, 5167)	0.06299789427872415
  (0, 5256)	0.159493076905487
  (0, 6299)	0.15590686996657177
  (0, 6317)	0.07421226313191584
  (0, 6988)	0.10724723400081589
  (0, 7389)	0.18637649960456082
  (0, 7400)	0.06718340158193163
  (0, 7477)	0.09610386835996934
  (0, 7611)	0.13315190145720263
  (0, 7702)	0.1379100299115112
  :	:
  (98, 1747)	0.21565539421544275
  (98, 3691)	0.39707468034611687
  (98, 7400)	0.14421300283519764
  (98, 9599)	0.18760674288717957
  (98, 9871)	0.23401749101245528
  (98, 10359)	0.3993074820508697
  